In [1]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nlp = spacy.load('en_core_web_sm')
file_path = 'climate_change_faqs.csv' 
faq_df = pd.read_csv(file_path)
questions = []
answers = []

current_question = None
for _, row in faq_df.iterrows():
    if row['text_type'] == 'q':
        current_question = row['faq']
    elif row['text_type'] == 'a' and current_question is not None:
        questions.append(current_question)
        answers.append(row['faq'])
        current_question = None

faq_pairs_df = pd.DataFrame({'Question': questions, 'Answer': answers})

In [3]:

def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)
faq_pairs_df['Processed_Question'] = faq_pairs_df['Question'].apply(preprocess_text)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(faq_pairs_df['Processed_Question'])
def get_best_match(query, faq_df, tfidf_matrix, vectorizer):
    processed_query = preprocess_text(query)
    query_vec = vectorizer.transform([processed_query])
    similarities = cosine_similarity(query_vec, tfidf_matrix)
    best_match_idx = similarities.argmax()
    best_question = faq_df['Question'].iloc[best_match_idx]
    best_answer = faq_df['Answer'].iloc[best_match_idx]
    return best_question, best_answer
stop_words = ['bye', 'goodbye', 'take care', 'exit']

In [4]:

def run_chatbot():
    print("Chatbot: Hello! Ask me anything about climate change.")
    while True:
        query = input("You: ").lower()
        if query in stop_words:
            print("Chatbot: Goodbye! Take care!")
            break
        else:
            best_question, best_answer = get_best_match(query, faq_pairs_df, tfidf_matrix, vectorizer)
            print(f"Chatbot:\n{best_answer}\n")

In [5]:

if __name__ == "__main__":
    run_chatbot()


Chatbot: Hello! Ask me anything about climate change.


You:  what is climate change ?


Chatbot:
Climate change (or global warming), is the process of our planet heating up.

Scientists estimate that since the Industrial Revolution, human activity has caused the Earth to warm by approximately 1°C. While that might not sound like much, it means big things for people and wildlife around the globe.

Unfortunately, rising temperatures don’t just mean that we’ll get nicer weather – if only! The changing climate will actually make our weather more extreme and unpredictable. 

As temperatures rise, some areas will get wetter and lots of animals (and humans!) could find they’re not able to adapt to their changing climate. 



You:  does deforestation alos contribute in climate change?


Chatbot:
Yes, deforestation contributes to climate change in a couple ways. Forests are one of our most important types of natural carbon storage, so when forests get cut down, they lose their ability to store the greenhouse gas. Burning trees release even more carbon into the atmosphere.

Forests are some of the best "natural climate solutions" we have on this planet. If we can slow or stop deforestation and manage natural land so that it is healthy, we could achieve up to one third of the emission reductions needed by 2030 to keep global temperatures from rising more than 2°C (3.6°C).

That’s the equivalent of the world putting a complete stop to burning oil.



You:  what action can we take to control it?


Chatbot:
Limiting global warming to 1.5°C above pre-industrial temperatures means CO2 emissions need to reach net zero by 2050. This will need to happen in every country, no matter how small or large. The nature of climate change means emissions have global impacts, and so a global collaborative response makes the challenge easier and cheaper to overcome in the long-run[1].  

However, there are also likely to be direct benefits from taking the lead in reducing emissions. By taking early and decisive action on climate change, the UK could gain a competitive edge in providing zero-carbon expertise, goods and services for other countries as they ramp up their action on climate change too. 



You:  exit


Chatbot: Goodbye! Take care!
